# Exercise 3: Clustering in Python


Imagine the following scenario:

*You are once again a Data Scientist at the fictitious company Adventure Works GmbH. After your successful analyses on the topics of Frequent Patterns and Classification, your bosses now assign you the task of dividing the products into groups of different profitability.*

*In discussions with your business administration colleagues, you learn that the decisive metrics here are probably the number of products actually sold and the profit per product (the sales price minus the production costs).*

*The colleagues from IT tell you that you will probably find the required data in the tables `Product` and `SalesOrderDetail`.*

In [ ]:
# Import required libraries
import os
import tempfile
import sqlite3
import urllib.request
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import Birch
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Create a temporary directory
dataset_folder = tempfile.mkdtemp()

# Build path to database
database_path = os.path.join(dataset_folder, "adventure-works.db")

# Get the database
urllib.request.urlretrieve(
    "https://github.com/FAU-CS6/KDD-Databases/raw/main/AdventureWorks/adventure-works.db",
    database_path,
)

# Open connection to the adventure-works.db
connection = sqlite3.connect(database_path)

<div class="alert alert-block alert-info">

**Task 1:** 
    
Cluster the products within the OLTP database of the fictitious Adventure Works GmbH according to their profitability. Furthermore, visualize the result.

</div>

In [ ]:
# Cluster the products based on their profitability (Code placeholder 01/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 02/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 03/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 04/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 05/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 06/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 07/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 08/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 09/10)

In [ ]:
# Cluster the products based on their profitability (Code placeholder 10/10)

The first step in this task is obviously to read the data from the database into a DataFrame and to take a first a look at the data at hand:

In [ ]:
# Load Product into a DataFrame and display the first ten rows
product_df = pd.read_sql_query("SELECT * FROM Product", connection)
product_df.head(10)

In [ ]:
# Load SalesOrderDetail into a DataFrame and display the first ten rows
sales_order_detail_df = pd.read_sql_query("SELECT * FROM SalesOrderDetail", connection)
sales_order_detail_df.head(10)

While getting to know the data, it is important to identify the relevant attributes. In our case we need:

- **Number of copies sold per product:**  
The table `SalesOrderDetail` contains information on how many products have been sold within that single order (`OrderQty`). If we sum up the `OrderQty` per product we get the number of copies sold per product.
- **Average profit per copy sold (per product):**  
First of all, it must be understood that the profit per sale can be calculated simply by subtracting the manufacturing cost (`StandardCost` in `Product`) from the actual selling price (`UnitPrice` in `SalesOrderDetail`). In this case, however, should not be summed up for a product, but the average should be determined.

These can either be created by preprocessing the initial DataFrames, or by using a simple SQL Statement for the aggregation. In this sample solution, we decided to do the latter:

In [ ]:
# Load `ProductID`,`ProfitPerUnit` and `OverallOrderQty` into a DataFrame
product_overview_df = pd.read_sql_query(
    "SELECT p.ProductID, AVG(sod.UnitPrice - p.StandardCost) AS ProfitPerUnit, SUM(sod.OrderQty) AS OverallOrderQty FROM Product p, SalesOrderDetail sod WHERE p.ProductID = sod.ProductID GROUP BY p.ProductID",
    connection,
)
product_overview_df.head(10)

Since the value range of `OverallOrderQty` goes from 4 to 8311 and the value range of `ProfitPerUnit` only goes from about -55 to about 1155, this dataset would not currently be a good fit for most clustering techniques. The `OverallOrderQty` would have a much higher influence in this case, which is why it makes sense to normalize the `product_overview_df` first:

In [ ]:
# Normalize the product_overview_df
min_max_scaler = MinMaxScaler()
product_overview_df[
    ["ProfitPerUnit", "OverallOrderQty"]
] = min_max_scaler.fit_transform(
    product_overview_df[["ProfitPerUnit", "OverallOrderQty"]]
)
product_overview_df.head(10)

This preprocessed DataFrame can be used for clustering. However, the question is which clustering method should be used. We will focus on K-Means, DBSCAN and BIRCH. All of these methods have been presented in the lecture and are implemented in scikit-learn:

In [ ]:
# Perform scikit-learn's K-means clustering on the dataset
kmeans = KMeans(n_clusters=6, n_init="auto").fit(
    product_overview_df[["ProfitPerUnit", "OverallOrderQty"]]
)

# Save the labels to a copy of the big_dataset to generate the equivalent of our clustered_big_dataset
clustered_product_overview_df = product_overview_df.copy()
clustered_product_overview_df["cluster"] = kmeans.labels_

# Print the result
plt.figure(figsize=(8, 8))
sns.scatterplot(
    x=clustered_product_overview_df["ProfitPerUnit"],
    y=clustered_product_overview_df["OverallOrderQty"],
    hue=clustered_product_overview_df["cluster"],
    palette="deep",
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

In [ ]:
# Perform scikit-learn's DBSCAN clustering on the dataset
dbscan = DBSCAN(eps=0.2, min_samples=5).fit(
    product_overview_df[["ProfitPerUnit", "OverallOrderQty"]]
)

# Save the labels to a copy of the big_dataset to generate the equivalent of our clustered_big_dataset
clustered_product_overview_df = product_overview_df.copy()
clustered_product_overview_df["cluster"] = dbscan.labels_

# Print the result
plt.figure(figsize=(8, 8))
sns.scatterplot(
    x=clustered_product_overview_df["ProfitPerUnit"],
    y=clustered_product_overview_df["OverallOrderQty"],
    hue=clustered_product_overview_df["cluster"],
    palette="deep",
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

In [ ]:
# Perform scikit-learn's BIRCH clustering on the dataset
birch = Birch(threshold=0.1, n_clusters=6).fit(
    product_overview_df[["ProfitPerUnit", "OverallOrderQty"]]
)

# Save the labels to a copy of the big_dataset to generate the equivalent of our clustered_big_dataset
clustered_product_overview_df = product_overview_df.copy()
clustered_product_overview_df["cluster"] = birch.labels_

# Print the result
plt.figure(figsize=(8, 8))
sns.scatterplot(
    x=clustered_product_overview_df["ProfitPerUnit"],
    y=clustered_product_overview_df["OverallOrderQty"],
    hue=clustered_product_overview_df["cluster"],
    palette="deep",
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

Of course, your fictitious bosses don't want to be presented with three different results from you. A clustering procedure should be chosen. For this purpose, it is useful to compare the results briefly:

- **K-means and BIRCH:**  
Both K-means and BIRCH produce quite similar results in this case. With our parameters we can interpret the found clusters the following way: 
  - Less interesting from a business perspective:
    - The cluster that does not bring much profit per unit and that was not sold frequently.
    - The cluster that does not bring much profit per unit, but was sold at least a little more often.
    - The cluster that brings a little more profit per unit, but which was not sold frequently.
  - Interesting from a business perspective:
    - The cluster that does not bring much profit per unit, but which was sold at least extremely often.
    - The cluster that is average both in terms of profit per unit and frequency of sales.
    - The cluster that brings extremely much profit, but which was hardly sold.

- **DBSCAN:**  
With our parameters we can interpret the found clusters the following way:
  - Less interesting from a business perspective:
    - Merged into one cluster
  - Interesting from a business perspective:
    - The cluster that does not bring much profit per unit, but which was sold at least extremely often.
    - The cluster that is average both in terms of profit per unit and frequency of sales.
    - The cluster that brings extremely much profit, but which was hardly sold.
    
Since DBSCAN merges the less interesting products into a cluster, it can be said that DBSCAN is probably the better choice here. This way, the focus in a presentation can be placed on the economically more interesting product clusters. 

However, it is of course not a big problem in this case if K-means or BIRCH are used, since one can at least argue that here a better distinction is made between completely uninteresting and at least somewhat interesting products.

Since the first scenario was quite straightforward, imagine a second scenario:

*As a Data Scientist who has already solved multiple different tasks for his bosses in the fictitious Adventure Works GmbH, you are immediately assigned another task. Your bosses want to make the sales team more efficient by assigning customers with similar product interests to the same employee.*

*In order to be able to carry out this reassignment, you are tasked with dividing the customers into 16 clusters (there are 16 sales persons in the company). This classification is to be based on the products that the customers have ordered in the past.*

*Via the IT department you learn that the customers can probably be found in the table ´Customer´. You will need to join the table `SalesOrderHeader` and then the table `SalesOrderDetail` to get information on the ordered ProductIDs per customers.*

<div class="alert alert-block alert-info">

**Task 2:**
    
Group the customers into 16 clusters by using K-means. You do not need to visualize the result.

</div>

In [ ]:
# Cluster the customers based on their interests (Code placeholder 01/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 02/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 03/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 04/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 05/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 06/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 07/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 08/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 09/10)

In [ ]:
# Cluster the customers based on their interests (Code placeholder 10/10)

The first step is again to load the data into a DataFrame:

In [ ]:
customer_purchases_df = pd.read_sql_query(
    "SELECT c.CustomerID, sod.ProductID, sod.OrderQty FROM Customer c JOIN SalesOrderHeader soh ON c.CustomerID = soh.CustomerID JOIN SalesOrderDetail sod ON sod.SalesOrderID = soh.SalesOrderID",
    connection,
    index_col="CustomerID",
)
customer_purchases_df.head(10)

After taking a first look at the available data, you should realize one thing above all:

The attribute `ProductID` is numeric. If the attribute were to be fed directly to the clustering, the numerical distance between two IDs would probably be used to calculate a distance between customers. 

A customer who bought the product with the ID `707` would thus be more dissimilar to a customer who bought the product with the ID `879` than to a customer who bought the product with the ID `712`. Usually, however, the distribution of product IDs is not based on similarity, but simply on the order in which the products were added to the catalog.

So for this task, we first need to put the data into a different format. Basically, we need to compare each customer's interest in each product individually. 

We have to choose between two different possibilities:

- **Concept 1:** Determine for each product if the customer has purchased the product (binary scale):

|            | Product 1 | Product 2 | Product 3 | Product 4 |
|------------|-----------|-----------|-----------|-----------|
| Customer 1 | 1         | 0         | 0         | 0         |
| Customer 2 | 1         | 1         | 0         | 0         |
| Customer 3 | 0         | 1         | 1         | 1         |


- **Concept 2:** Determine sum of copies purchased for each product purchased (continuous scale): 

|            | Product 1 | Product 2 | Product 3 | Product 4 |
|------------|-----------|-----------|-----------|-----------|
| Customer 1 | 236       | 0         | 0         | 0         |
| Customer 2 | 1         | 199       | 0         | 0         |
| Customer 3 | 0         | 199       | 5         | 1         |

In the first variant, `Customer 1` and `Customer 2` would be most similar. In the second variant, the interests of `Customer 2` and `Customer 3` would be most similar.

Since both options have advantages and disadvantages, but we cannot discuss with our fictitious bosses which option is better for our company, we had to decide for ourselves and in this sample solution we simply have opted for concept 2.

Of course, we also need the quantity of all purchases in the DataFrame, which we do using SQL:

In [ ]:
# Load a DataFrame that contains the total copies purchased per customer and product
customer_interests_df = pd.read_sql_query(
    "SELECT c.CustomerID, sod.ProductID, SUM(sod.OrderQty) AS TotalOrderQty FROM Customer c JOIN SalesOrderHeader soh ON c.CustomerID = soh.CustomerID JOIN SalesOrderDetail sod ON sod.SalesOrderID = soh.SalesOrderID GROUP BY c.CustomerID, sod.ProductID",
    connection,
)
customer_interests_df.tail(10)

This DataFrame can then be pivoted in order to arrive at a DataFrame corresponding to the above concept:

In [ ]:
# Pivot the DataFrame
customer_interests_pivot_df = customer_interests_df.pivot(
    index="CustomerID", columns="ProductID", values="TotalOrderQty"
).fillna(0)
customer_interests_pivot_df.tail(10)

It is now prepared to be used in clustering:


In [ ]:
# Perform scikit-learn's K-means clustering on the dataset
kmeans = KMeans(n_clusters=16, n_init="auto").fit(customer_interests_pivot_df)

# Save the labels to a copy of the big_dataset to generate the equivalent of our clustered_big_dataset
clustered_customer_interests_pivot_df = customer_interests_pivot_df.copy()
clustered_customer_interests_pivot_df["cluster"] = kmeans.labels_

# Print the resulting DataFrame
clustered_customer_interests_pivot_df.tail(10)

Even if this leads a clustering result, you might want to take a look at the value distribution in these clusters:

In [ ]:
# Get the count of customers per cluster
clustered_customer_interests_pivot_df[["cluster"]].value_counts()

A very unbalanced distribution can be seen, indicating that these clusters should definitely not be used in this way to assign customers to the 16 sales persons. If you try the different clustering methods implemented scikit-learn, you will notice that none of the methods changes this imbalance decisively. 

This is simply because clustering is not designed to achieve (roughly) equal cluster sizes. There are ideas here on how to get around this (e.g., calculate significantly more clusters and then merge each of these with neighbors until about the required size is reached), but these new groups would then not necessarily only contain customers with similar interests. 

It would make more sense here to approach the fictitious bosses again and tell them that similarity of interests is probably not the best criterion for dividing customers among sales persons. 

Since it is of course a shame to end an exercise sheet with a perceived failure: 
There are some interesting things that can be concluded from the identified clusters. We now know that the majority of the customer base seems to share similar interests. This can be pitched to the management to further specialize the focus of the company. Even failures in data science sometimes contain new insights, you just have to be open enough to discover them.